##### Dependencies
____

In [18]:
import numpy as np
import pandas as pd
import sqlalchemy as db
import datetime
import random

from faker import Faker

##### Database connection
____

In [19]:
g_params = {
    'dbuser'   : 'csc4710',
    'dbpwd'    : 'password',
    'dbserver' : 'localhost',
    'dbport'   : 3306,
    'dbname'   : 'CSC4710'
}

disk_engine = db.create_engine(f'mysql+pymysql://{g_params["dbuser"]}:{g_params["dbpwd"]}@{g_params["dbserver"]}:{g_params["dbport"]}/{g_params["dbname"]}', echo=False)
disk_engine.connect().connection.ping()

##### Load Address Dataframe
____

In [20]:
address = pd.read_sql_table(table_name='address', con=disk_engine)

address.head(5)

,id,street,city,state,zipcode,latitude,longitude
0,1,2175 PIEDMONT RD NE,Atlanta,GA,30324-4128,33.817098,-84.366785
1,2,306 LUCKIE ST NW,Atlanta,GA,30313-1706,33.762858,-84.395977
2,3,227 WELLINGTON ST SW,Atlanta,GA,30314-2240,33.749008,-84.438294
3,4,1125 CASCADE CIR SW @ZONE 4,Atlanta,GA,30311-2817,33.724948,-84.449931
4,5,469 FAIR ST SW,Atlanta,GA,30313-1203,33.748725,-84.403206


##### Load Employee Dataframe
____

In [21]:
employees = pd.read_sql_table(table_name='employee', con=disk_engine)

cond = employees['supervisor'].isna()

employees = employees[cond]

employees

,id,firstname,lastname,ssn,holiday_status,phone,address,supervisor
0,1,Daniel,Mejia,749-96-2475,off,354-818-0021x8551,41,NaN
1,2,Richard,Johnson,660-08-9388,off,108.889.4800x2458,19,NaN
2,3,Kristi,Fernandez,869-09-5521,off,3246551265,28,NaN
3,4,Tyler,Freeman,270-34-6727,off,001-806-050-7647x41840,9,NaN
4,5,Kaitlin,Harrell,509-08-5361,off,0474937466,59,NaN


##### Add Restaurant Types
____

In [22]:
categories = ['Fast food', 'Fast casual', 'Casual dining', 'Premium casual', 'Family style', 'Fine dining']
restaurant_types = []

for item in categories:    
    sql = "INSERT INTO restaurant_types(category, description) VALUES (%s, %s)"
    values = (item, f'{item} description') 
    
    restaurant_types.append(disk_engine.execute(sql, values).lastrowid)

##### Add Restaurant
____

In [23]:
total = 1
faker = Faker(locale='en_US')

for i in range(1, total+1):
    rest_type   = random.sample(restaurant_types, 1)[0]
    manager     = int(employees['id'].sample(n=1).values[0])
    rest_addr   = int(address['id'].sample(n=1).values[0])
    name        = f'Auto generated {rest_type}'
    phone       = faker.phone_number()
    website     = faker.url()
    status      = 'open'
    open_at     = '09:00:00'
    close_at    = '22:00:00'
    day_of_week = 127                      #1 = Sunday, 2 = Monday, 4 = Tuesday, 8 = Wednesday, 16 = Thursday, 32 = Friday, 64 = Saturday
    
    sql = "INSERT INTO restaurant(type, manager, address, name, phone, website, status, open, close, day_of_week) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (rest_type, manager, rest_addr, name, phone, website, status, open_at, close_at, day_of_week) 
    
    disk_engine.execute(sql, values)    